In [2]:
import pandas as pd
import numpy as np

In [27]:
durationDF = pd.read_csv('data/Durations.csv').set_index("ContestId")

## Kalman AB Processing

In [110]:
abDF = pd.read_csv('data/KF_Values.csv').set_index("ContestId")
df = pd.merge(durationDF, abDF, on='ContestId', how='left')
df = df.replace('TooShort', 0).replace('NaN', 0).replace(np.nan, 0)

In [111]:
kfCols = []
for col in df.columns:
    if(col[0]=='A'):
        v = col[1:]
        newCol = 'Pred'+v
        print("Processing", v)
        df[newCol] = df['A'+v].astype(float)*np.exp(df['B'+v].astype(float)*df['Duration'])
        kfCols.append(newCol)

Processing v11
Processing v12
Processing v13
Processing v14
Processing v15
Processing v1
Processing v10
Processing v2
Processing v3
Processing v4
Processing v5
Processing v6
Processing v7
Processing v8
Processing v9


In [112]:
# This needs to 
# df[kfCols].to_csv('data/KF_Preds.csv')

In [113]:
df.to_csv('data/KF_Preds.csv')

### Scaled KF

In [106]:
abDF = pd.read_csv('data/KF_Scaled_Values.csv').drop(columns=['Unnamed: 0']).set_index("ContestId")
df = abDF.replace('TooShort', 0).replace('NaN', 0).replace(np.nan, 0)

In [107]:
kfCols = [kfCols]
for col in df.columns:
    if(col[0]=='A'):
        v = col[1:]
        newCol = 'Pred'+v
        print("Processing", v)
        df[newCol] = df['A'+v].astype(float)*np.exp(df['B'+v].astype(float)*100)
        kfCols.append(newCol)

Processing v1
Processing v10
Processing v11
Processing v12
Processing v13
Processing v14
Processing v15
Processing v2
Processing v3
Processing v4
Processing v5
Processing v6
Processing v7
Processing v8
Processing v9


In [108]:
# df[kfCols].to_csv('data/KF_Scaled_Preds.csv')

In [109]:
df.to_csv('data/KF_Scaled_Preds.csv')

## Linear Regression AB Processing

In [94]:
LRDF = pd.read_csv("data/LR_Values.csv")

In [95]:
df = pd.merge(durationDF, LRDF, on='ContestId', how='left')
df = df.replace('TooShort', 0)

In [96]:
lrCols = []
for col in df.columns:
    if(col[::-1][0]=='A'):
        v = col[:-1]
        newCol = 'Pred'+v

        print("Processing", v)
        df[newCol] = df[v+'A'].astype(float)*np.exp(df[v+'B'].astype(float)*df['Duration'])
        lrCols.append(newCol)

Processing f10
Processing f11
Processing f12
Processing f13
Processing f14
Processing f15
Processing f1
Processing f2
Processing f3
Processing f4
Processing f5
Processing f6
Processing f7
Processing f8
Processing f9


In [97]:
df = df.replace(np.nan, 0.)

In [99]:
df.to_csv('data/LR_Preds.csv')

### Scaled LR

In [100]:
LRDF = pd.read_csv("data/LR_Scaled_Values.csv").set_index('ContestId')

In [101]:
df = LRDF.replace('TooShort', 0).copy()

In [102]:
lrCols = []
for col in df.columns:
    if(col[::-1][0]=='A'):
        v = col[:-1]
        print("Processing", v)
        newCol = 'Pred'+v
        df[newCol] = df[v+'A'].astype(float)*np.exp(df[v+'B'].astype(float)*100)
        lrCols.append(newCol)

Processing f10
Processing f11
Processing f12
Processing f13
Processing f14
Processing f15
Processing f1
Processing f2
Processing f3
Processing f4
Processing f5
Processing f6
Processing f7
Processing f8
Processing f9


In [103]:
df = df.replace(np.nan, 0.)

In [104]:
df.to_csv('data/LR_Scaled_Preds.csv')